# Exploring EAZY results

* Load and print results
* Label color image with id numbers and photo-z's
* Galaxy card compiling color image and plots

## Imports

In [1]:
import eazy
import eazy.hdf5

In [2]:
from astropy.table import Table
import astropy.units as u
import astropy

import numpy as np
from glob import glob
import string
from importlib import reload

import os
from os.path import expanduser
home = expanduser("~")

In [3]:
# to show segmentation map
import photutils

In [4]:
# for labeling color image
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.wcs as wcs

In [5]:
# for labeling color image
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
from PIL.PngImagePlugin import PngInfo
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000  # avoid decompression bomb error when loading large images

In [6]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker
#%matplotlib inline   # non-interactive (easier for notebook scrolling)
%matplotlib notebook
plt.style.use(os.path.join(home, 'p', 'matplotlibrc.txt')) # https://matplotlib.org/tutorials/introductory/customizing.html
#plt.style.use('https://www.stsci.edu/~dcoe/matplotlibrc.txt') # https://matplotlib.org/tutorials/introductory/customizing.html
mpl_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

## Helper functions

In [7]:
def extract_id(cat, id, idlabel='id'): # choose_object select_object
    # Note input id must be same format as cat[idlabel] (int or string)
    # to create an array with mostly False entries and True for id
    # cat['id'] dtype='int64'
    # cat['id'] == 3: [False, False, True, False...]
    # cat['id'] == '3': False
    #
    # obj:          astropy.table.table.Table
    # obj[0]:       astropy.table.row.Row
    # obj['id']:    astropy.table.column.Column
    # obj[0]['id']: numpy.int64
    duck_duck_goose = cat[idlabel] == int(id)
    if len(duck_duck_goose):
        obj = cat[duck_duck_goose]
    return obj[0]

In [8]:
def between(lo, x, hi, tol=0):
    return (lo-tol < x) & (x < hi+tol)

In [9]:
# Format exponential ticks to read as simple numbers unless too big / small
def fmtexp(x, pos):
    if between(1e-4, x, 1e4):
        s = '%g' % x
    else:
        p = int(np.log10(x))
        d = int(np.round(x / 10**p))
        #print d
        s = ''
        if d > 1:
            s += '%d$\\times$' % d
    
        s += '10${\\mathdefault{^{%d}}}$' % p
    return s

# Conversions for second axis
def AB2uJy(mAB):
    m = mAB * u.ABmag
    f = m.to(u.uJy)
    return f.value

def uJy2AB(F_uJy):
    f = F_uJy * u.uJy
    m = f.to(u.ABmag)
    return m.value

def AB2nJy(mAB):
    m = mAB * u.ABmag
    f = m.to(u.nJy)
    return f.value

def nJy2AB(F_nJy):
    f = F_nJy * u.nJy
    m = f.to(u.ABmag)
    return m.value

# Combined output file: photometry + photo-z's

In [10]:
if 0:
    inroot = 'sunrise-grizli-v2-fix'
    catalog_file = inroot + '_phot.fits'
    catalog = Table.read(catalog_file)
    catalog[:2]

# Load photometry catalog

In [11]:
field = 'macs0647'

In [12]:
phot_cat = astropy.io.ascii.read(field+'_phot-eazy.ecsv')
phot_cat[:2]

id,ra,dec,x,y,z_spec,f105w_flux,f105w_fluxerr,f110w_flux,f110w_fluxerr,f115w_flux,f115w_fluxerr,f125w_flux,f125w_fluxerr,f140w_flux,f140w_fluxerr,f150w_flux,f150w_fluxerr,f160w_flux,f160w_fluxerr,f200w_flux,f200w_fluxerr,f275wu_flux,f275wu_fluxerr,f277w_flux,f277w_fluxerr,f336wu_flux,f336wu_fluxerr,f356w_flux,f356w_fluxerr,f390wu_flux,f390wu_fluxerr,f435w_flux,f435w_fluxerr,f444w_flux,f444w_fluxerr,f475w_flux,f475w_fluxerr,f555w_flux,f555w_fluxerr,f606w_flux,f606w_fluxerr,f625w_flux,f625w_fluxerr,f775w_flux,f775w_fluxerr,f814w_flux,f814w_fluxerr,f850lp_flux,f850lp_fluxerr,nusefilt,z_ml,z_ml_chi2,z_ml_risk,lc_min,lc_max,z_phot,z_phot_chi2,z_phot_risk,z_min_risk,min_risk,z_raw_chi2,raw_chi2,z025,z160,z500,z840,z975,restU,restU_err,restB,restB_err,restV,restV_err,restJ,restJ_err,dL,mass,sfr,Lv,LIR,energy_abs,Lu,Lj,L1400,L2800,LHa,LOIII,LHb,LOII,MLv,Av,lwAgeV,mass_p,sfr_p,Lv_p,LIR_p,energy_abs_p,Lu_p,Lj_p,L1400_p,L2800_p,LHa_p,LOIII_p,LHb_p,LOII_p,Av_p,ssfr_p,rest120,rest120_err,rest121,rest121_err,rest156,rest156_err,rest157,rest157_err,rest158,rest158_err,rest159,rest159_err,rest160,rest160_err,rest414,rest414_err,rest415,rest415_err,rest416,rest416_err
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Mpc,solMass,solMass / yr,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solMass / solLum,,Gyr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int32,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,102.015220,70.161446,4098.274752800464,1.999824071826548,-1.0,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0,-1.0,-1.0,-1.0,27082581599519.1,0.0,-1.0,-1.0,-1.0,0.01,0.0,0.01,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,0.0e+00,-9.00e+29,-900000000000000046043660025856.000,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.000 .. -900000000000000046043660025856.000,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.00 .. -900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000
2,101.948789,70.161459,8156.305414383205,1.9979236794378306,-1.0,-99.000,-99.000,-99.000,-99.

In [13]:
# Reconstitute filters
filters = []
for label in phot_cat.columns:
    if label.endswith('_flux'):
        filters.append(label[:-5])
        
filters

['f105w',
 'f110w',
 'f115w',
 'f125w',
 'f140w',
 'f150w',
 'f160w',
 'f200w',
 'f275wu',
 'f277w',
 'f336wu',
 'f356w',
 'f390wu',
 'f435w',
 'f444w',
 'f475w',
 'f555w',
 'f606w',
 'f625w',
 'f775w',
 'f814w',
 'f850lp']

# Load EAZY results

In [14]:
inroot = field + '-grizli-v4.0-fix'

In [16]:
eazy_file = inroot + '.eazypy.zout.fits.gz'
eazy_results = Table.read(eazy_file)
eazy_results[:2]

id,ra,dec,z_spec,nusefilt,z_ml,z_ml_chi2,z_ml_risk,lc_min,lc_max,z_phot,z_phot_chi2,z_phot_risk,z_min_risk,min_risk,z_raw_chi2,raw_chi2,z025,z160,z500,z840,z975,restU,restU_err,restB,restB_err,restV,restV_err,restJ,restJ_err,dL,mass,sfr,Lv,LIR,energy_abs,Lu,Lj,L1400,L2800,LHa,LOIII,LHb,LOII,MLv,Av,lwAgeV,mass_p,sfr_p,Lv_p,LIR_p,energy_abs_p,Lu_p,Lj_p,L1400_p,L2800_p,LHa_p,LOIII_p,LHb_p,LOII_p,Av_p,ssfr_p,rest120,rest120_err,rest121,rest121_err,rest156,rest156_err,rest157,rest157_err,rest158,rest158_err,rest159,rest159_err,rest160,rest160_err,rest414,rest414_err,rest415,rest415_err,rest416,rest416_err
,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,Mpc,solMass,solMass / yr,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solLum,solMass / solLum,,Gyr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int32,float64,float64,float64,int64,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float64[5],float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,102.01521994463253,70.16144628801996,-1.0,0,-1.0,-1.0,-1.0,27082581599519.1,0.0,-1.0,-1.0,-1.0,0.01,0.0,0.01,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,0.0e+00,-9.00e+29,-900000000000000046043660025856.000,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.000 .. -900000000000000046043660025856.000,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.00 .. -900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000
2,101.94878938557765,70.16145877544555,-1.0,0,-1.0,-1.0,-1.0,27082581599519.1,0.0,-1.0,-1.0,-1.0,0.01,0.0,0.01,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,0.0e+00,-9.00e+29,-900000000000000046043660025856.000,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-9.00e+29,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.000 .. -900000000000000046043660025856.000,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-9.00e+29 .. -9.00e+29,-900000000000000046043660025856.00 .. -900000000000000046043660025856.00,-9.00e+29 .. -9.00e+29,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000,-99.000,0.000


In [17]:
eazy_phot = Table.read(inroot + '_phot_apcorr.fits.gz')
#eazy_phot = Table.read(inroot + '.eazypy.data.fits')
eazy_phot[:2]

id,thresh,npix,tnpix,xmin,xmax,ymin,ymax,x,y,x2_image,y2_image,xy_image,errx2,erry2,errxy,a_image,b_image,theta_image,cxx_image,cyy_image,cxy_image,cflux,flux,cpeak,peak,xcpeak,ycpeak,xpeak,ypeak,flag,x_image,y_image,number,ra,dec,x_world,y_world,flux_iso,fluxerr_iso,area_iso,mag_iso,kron_radius,kron_rcirc,flux_auto,fluxerr_auto,bkg_auto,flag_auto,area_auto,flux_radius_flag,flux_radius_20,flux_radius,flux_radius_90,tot_corr,mag_auto,magerr_auto,flux_aper_0,fluxerr_aper_0,flag_aper_0,bkg_aper_0,mask_aper_0,flux_aper_1,fluxerr_aper_1,flag_aper_1,bkg_aper_1,mask_aper_1,flux_aper_2,fluxerr_aper_2,flag_aper_2,bkg_aper_2,mask_aper_2,f105w_flux_aper_0,f105w_fluxerr_aper_0,f105w_flag_aper_0,f105w_bkg_aper_0,f105w_mask_aper_0,f105w_flux_aper_1,f105w_fluxerr_aper_1,f105w_flag_aper_1,f105w_bkg_aper_1,f105w_mask_aper_1,f105w_flux_aper_2,f105w_fluxerr_aper_2,f105w_flag_aper_2,f105w_bkg_aper_2,f105w_mask_aper_2,f105w_tot_corr,f110w_flux_aper_0,f110w_fluxerr_aper_0,f110w_flag_aper_0,f110w_bkg_aper_0,f110w_mask_aper_0,f110w_flux_aper_1,f110w_fluxerr_aper_1,f110w_flag_aper_1,f110w_bkg_aper_1,f110w_mask_aper_1,f110w_flux_aper_2,f110w_fluxerr_aper_2,f110w_flag_aper_2,f110w_bkg_aper_2,f110w_mask_aper_2,f110w_tot_corr,f115w_flux_aper_0,f115w_fluxerr_aper_0,f115w_flag_aper_0,f115w_bkg_aper_0,f115w_mask_aper_0,f115w_flux_aper_1,f115w_fluxerr_aper_1,f115w_flag_aper_1,f115w_bkg_aper_1,f115w_mask_aper_1,f115w_flux_aper_2,f115w_fluxerr_aper_2,f115w_flag_aper_2,f115w_bkg_aper_2,f115w_mask_aper_2,f115w_tot_corr,f125w_flux_aper_0,f125w_fluxerr_aper_0,f125w_flag_aper_0,f125w_bkg_aper_0,f125w_mask_aper_0,f125w_flux_aper_1,f125w_fluxerr_aper_1,f125w_flag_aper_1,f125w_bkg_aper_1,f125w_mask_aper_1,f125w_flux_aper_2,f125w_fluxerr_aper_2,f125w_flag_aper_2,f125w_bkg_aper_2,f125w_mask_aper_2,f125w_tot_corr,f140w_flux_aper_0,f140w_fluxerr_aper_0,f140w_flag_aper_0,f140w_bkg_aper_0,f140w_mask_aper_0,f140w_flux_aper_1,f140w_fluxerr_aper_1,f140w_flag_aper_1,f140w_bkg_aper_1,f140w_mask_aper_1,f140w_flux_aper_2,f140w_fluxerr_aper_2,f140w_flag_aper_2,f140w_bkg_aper_2,f140w_mask_aper_2,f140w_tot_corr,f150w_flux_aper_0,f150w_fluxerr_aper_0,f150w_flag_aper_0,f150w_bkg_aper_0,f150w_mask_aper_0,f150w_flux_aper_1,f150w_fluxerr_aper_1,f150w_flag_aper_1,f150w_bkg_aper_1,f150w_mask_aper_1,f150w_flux_aper_2,f150w_fluxerr_aper_2,f150w_flag_aper_2,f150w_bkg_aper_2,f150w_mask_aper_2,f150w_tot_corr,f160w_flux_aper_0,f160w_fluxerr_aper_0,f160w_flag_aper_0,f160w_bkg_aper_0,f160w_mask_aper_0,f160w_flux_aper_1,f160w_fluxerr_aper_1,f160w_flag_aper_1,f160w_bkg_aper_1,f160w_mask_aper_1,f160w_flux_aper_2,f160w_fluxerr_aper_2,f160w_flag_aper_2,f160w_bkg_aper_2,f160w_mask_aper_2,f160w_tot_corr,f200w_flux_aper_0,f200w_fluxerr_aper_0,f200w_flag_aper_0,f200w_bkg_aper_0,f200w_mask_aper_0,f200w_flux_aper_1,f200w_fluxerr_aper_1,f200w_flag_aper_1,f200w_bkg_aper_1,f200w_mask_aper_1,f200w_flux_aper_2,f200w_fluxerr_aper_2,f200w_flag_aper_2,f200w_bkg_aper_2,f200w_mask_aper_2,f200w_tot_corr,f275wu_flux_aper_0,f275wu_fluxerr_aper_0,f275wu_flag_aper_0,f275wu_bkg_aper_0,f275wu_mask_aper_0,f275wu_flux_aper_1,f275wu_fluxerr_aper_1,f275wu_flag_aper_1,f275wu_bkg_aper_1,f275wu_mask_aper_1,f275wu_flux_aper_2,f275wu_fluxerr_aper_2,f275wu_flag_aper_2,f275wu_bkg_aper_2,f275wu_mask_aper_2,f275wu_tot_corr,f277w_flux_aper_0,f277w_fluxerr_aper_0,f277w_flag_aper_0,f277w_bkg_aper_0,f277w_mask_aper_0,f277w_flux_aper_1,f277w_fluxerr_aper_1,f277w_flag_aper_1,f277w_bkg_aper_1,f277w_mask_aper_1,f277w_flux_aper_2,f277w_fluxerr_aper_2,f277w_flag_aper_2,f277w_bkg_aper_2,f277w_mask_aper_2,f277w_tot_corr,f336wu_flux_aper_0,f336wu_fluxerr_aper_0,f336wu_flag_aper_0,f336wu_bkg_aper_0,f336wu_mask_aper_0,f336wu_flux_aper_1,f336wu_fluxerr_aper_1,f336wu_flag_aper_1,f336wu_bkg_aper_1,f336wu_mask_aper_1,f336wu_flux_aper_2,f336wu_fluxerr_aper_2,f336wu_flag_aper_2,f336wu_bkg_aper_2,f336wu_mask_aper_2,f336wu_tot_corr,f356w_flux_aper_0,f356w_fluxerr_aper_0,f356w_flag_aper_0,f356w_bkg_aper_0,f356w_mask_aper_0,f356w_flux_aper_1,f356w_fluxerr_aper_1,f

In [ ]:
#eazy_data = fits.open(inroot + '.eazypy.data.fits')
#eazy_data.info()

In the .h5 file, EAZY saved local paths to templates and FILTER.RES.latest.   
Link to them on your machine so we can find them now! For example:

ln /Users/dcoe/miniconda3/envs/erophot/lib/python3.10/site-packages/eazy/data/templates   
ln /Users/dcoe/miniconda3/envs/erophot/lib/python3.10/site-packages/eazy/data/filters/FILTER.RES.latest

In [18]:
h5file  = inroot + '.eazypy.h5'
eazy_run = eazy.hdf5.initialize_from_hdf5(h5file=h5file)

Read default param file: /Users/dcoe/miniconda3/envs/dja/lib/python3.11/site-packages/eazy/data/zphot.param.default
CATALOG_FILE is a table
   >>> NOBJ = 5500
f105w_tot_1 f105w_etot_1 (202): hst/wfc3/IR/f105w.dat
f110w_tot_1 f110w_etot_1 (241): hst/wfc3/IR/f110w.dat
f115w_tot_1 f115w_etot_1 (364): jwst_nircam_f115w
f125w_tot_1 f125w_etot_1 (203): hst/wfc3/IR/f125w.dat
f140w_tot_1 f140w_etot_1 (204): hst/wfc3/IR/f140w.dat
f150w_tot_1 f150w_etot_1 (365): jwst_nircam_f150w
f160w_tot_1 f160w_etot_1 (205): hst/wfc3/IR/f160w.dat
f200w_tot_1 f200w_etot_1 (366): jwst_nircam_f200w
f275wu_tot_1 f275wu_etot_1 (208): hst/wfc3/UVIS/f275w.dat
f277w_tot_1 f277w_etot_1 (375): jwst_nircam_f277w
f336wu_tot_1 f336wu_etot_1 (209): hst/wfc3/UVIS/f336w.dat
f356w_tot_1 f356w_etot_1 (376): jwst_nircam_f356w
f390wu_tot_1 f390wu_etot_1 (210): hst/wfc3/UVIS/f390w.dat
f435w_tot_1 f435w_etot_1 (233): hst/ACS_update_sep07/wfc_f435w_t81.dat
f444w_tot_1 f444w_etot_1 (377): jwst_nircam_f444w
f475w_tot_1 f475w_etot_1 (

TemplateGrid: user-provided tempfilt_data
Process templates: 0.490 s


294it [00:00, 754.35it/s]


h5: read corr_sfhz_13_bin0_av0.01.fits
h5: read corr_sfhz_13_bin0_av0.25.fits
h5: read corr_sfhz_13_bin0_av0.50.fits
h5: read corr_sfhz_13_bin0_av1.00.fits
h5: read corr_sfhz_13_bin1_av0.01.fits
h5: read corr_sfhz_13_bin1_av0.25.fits
h5: read corr_sfhz_13_bin1_av0.50.fits
h5: read corr_sfhz_13_bin1_av1.00.fits
h5: read corr_sfhz_13_bin2_av0.01.fits
h5: read corr_sfhz_13_bin2_av0.50.fits
h5: read corr_sfhz_13_bin2_av1.00.fits
h5: read corr_sfhz_13_bin3_av0.01.fits
h5: read corr_sfhz_13_bin3_av0.50.fits
h5: read 4590.fits
fit_best: 0.5 s (n_proc=1,  NOBJ=4843)


In [19]:
eazy_run.cat[:2]

id,ra,dec,z_spec,f105w_tot_1,f105w_etot_1,f110w_tot_1,f110w_etot_1,f115w_tot_1,f115w_etot_1,f125w_tot_1,f125w_etot_1,f140w_tot_1,f140w_etot_1,f150w_tot_1,f150w_etot_1,f160w_tot_1,f160w_etot_1,f200w_tot_1,f200w_etot_1,f275wu_tot_1,f275wu_etot_1,f277w_tot_1,f277w_etot_1,f336wu_tot_1,f336wu_etot_1,f356w_tot_1,f356w_etot_1,f390wu_tot_1,f390wu_etot_1,f435w_tot_1,f435w_etot_1,f444w_tot_1,f444w_etot_1,f475w_tot_1,f475w_etot_1,f555w_tot_1,f555w_etot_1,f606w_tot_1,f606w_etot_1,f625w_tot_1,f625w_etot_1,f775w_tot_1,f775w_etot_1,f814w_tot_1,f814w_etot_1,f850lp_tot_1,f850lp_etot_1
int32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,102.015220,70.161446,-1.00000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000
2,101.948789,70.161459,-1.00000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000


In [20]:
catalog = eazy_run.cat[:]

if 1:
    for oldlabel in catalog.columns[:]:
        newlabel = oldlabel.replace('_tot_1', '_flux')
        newlabel = newlabel.replace('_etot_1', '_fluxerr')
        newlabel = newlabel.replace('-clear', '')
        catalog.rename_column(oldlabel, newlabel)
        #print(newlabel)

In [21]:
for label in 'x y'.split():
    catalog[label] = 2 * eazy_phot[label]  # 40mas -> 20mas

In [22]:
np.max(catalog['z_spec'])

3.15521

In [23]:
#catalog.remove_column('z_spec')

In [24]:
#catalog[:2]

In [25]:
# Move x,y forward in table
columns = list(catalog.columns)
columns = columns[:3] + columns[-2:] + columns[3:-2]
catalog = catalog[columns]

In [26]:
catalog[:2]

id,ra,dec,x,y,z_spec,f105w_flux,f105w_fluxerr,f110w_flux,f110w_fluxerr,f115w_flux,f115w_fluxerr,f125w_flux,f125w_fluxerr,f140w_flux,f140w_fluxerr,f150w_flux,f150w_fluxerr,f160w_flux,f160w_fluxerr,f200w_flux,f200w_fluxerr,f275wu_flux,f275wu_fluxerr,f277w_flux,f277w_fluxerr,f336wu_flux,f336wu_fluxerr,f356w_flux,f356w_fluxerr,f390wu_flux,f390wu_fluxerr,f435w_flux,f435w_fluxerr,f444w_flux,f444w_fluxerr,f475w_flux,f475w_fluxerr,f555w_flux,f555w_fluxerr,f606w_flux,f606w_fluxerr,f625w_flux,f625w_fluxerr,f775w_flux,f775w_fluxerr,f814w_flux,f814w_fluxerr,f850lp_flux,f850lp_fluxerr
int32,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,102.015220,70.161446,4098.274752800464,1.999824071826548,-1.00000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000
2,101.948789,70.161459,8156.305414383205,1.9979236794378306,-1.00000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000


# Show results

In [29]:
# 1395, 1715, 1742, 3989
id = 1395
fig, data = eazy_run.show_fit(id, xlim=[0.4, 5], show_fnu=True, logpz=True, show_prior=True) #, zshow=1.62)
fig.savefig('eazy_SED-Pz_%d.png' % id)

<IPython.core.display.Javascript object>

/var/folders/y3/vsjr6j554mnfpgy_m1xrnckm0003tz/T/ipykernel_78165/701953445.py:4: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.savefig('eazy_SED-Pz_%d.png' % id)


In [33]:
ids = np.loadtxt('highzids_v4_final.txt')
ids

array([5156., 3035.,  486., 3186., 3568.,  986., 4865., 4915.,  946.,
       1882., 4514., 3493., 1559., 5332., 3643., 1139., 4920., 3138.,
       1378., 3989.,  198.,  661., 4893., 4219., 1751., 4914., 4838.,
       5420., 2876., 1243., 5352., 4775.,  116., 4186., 1924., 1395.,
       1715., 2019., 3533., 3754., 5094., 3308., 1912., 1714., 3208.,
       3090., 1742.,  926., 5010., 1944.])

In [36]:
for id in ids:
    fig, data = eazy_run.show_fit(id, xlim=[0.4, 5], show_fnu=True, logpz=True, show_prior=True) #, zshow=1.62)
    fig.savefig('eazy/eazy_SED-Pz_v4_%d.png' % id)

<IPython.core.display.Javascript object>

/var/folders/y3/vsjr6j554mnfpgy_m1xrnckm0003tz/T/ipykernel_78165/4010183718.py:3: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.savefig('eazy/eazy_SED-Pz_v4_%d.png' % id)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
fig, data = eazy_run.show_fit(id, xlim=[0.4, 5], show_fnu=True, maglim=(25,17), logpz=True, show_prior=True)

In [ ]:
fig, data = eazy_run.show_fit(id, xlim=[0.4, 5], show_fnu=True, logpz=True, zshow=2)

# Look at another

In [ ]:
# https://relics.stsci.edu/data/macs0647/JWST/EAZY/eazy_3539.png
id = 3539 # z ~ 11 bright blue
fig, data = eazy_run.show_fit(id, xlim=[0.4, 5], show_fnu=True, logpz=True, show_prior=True)

In [ ]:
fig, data = eazy_run.show_fit(id, xlim=[0.4, 5], show_fnu=True, maglim=[30,25], logpz=True)

In [ ]:
# https://relics.stsci.edu/data/macs0647/JWST/EAZY/eazy_3353.png
id = 3533 # z ~ 11 bright blue arc
fig, data = eazy_run.show_fit(id, xlim=[0.4, 5], show_fnu=True, logpz=True, show_prior=True)

In [ ]:
SNR = data['fobs'] / data['efobs']
for i, filt in enumerate(filters):
    print(filt.ljust(7), '% 5.1f' % SNR[i])

# Example outputs

In [ ]:
extract_id(phot_cat, id)

In [ ]:
(0.0828 * u.uJy).to(u.ABmag)

In [ ]:
obj['sfr']  # solMass / yr

In [ ]:
obj['mass']  # solMass

In [ ]:
obj['sfr'] / obj['mass'] * 1e9  # sSFR: specific star formation rate (/ Gyr)

In [ ]:
obj['ssfr_p'] * 1e9  # percentiles

# Find object in catalog

In [ ]:
def roundint(x):
    return np.round(x).astype(int)

In [ ]:
def find_object_in_catalog(RA, Dec):
    object_coordinates = SkyCoord(ra=RA*u.deg, dec=Dec*u.deg)
    
    # First look in segmentation map:
    x, y = image_wcs.world_to_pixel(object_coordinates)
    x = roundint(x)
    y = roundint(y)
    id = segm.data[y,x]
    
    # If nothing there, check for nearby object
    if not id:
        catalog_coordinates = SkyCoord(ra=eazy_results['ra'], dec=eazy_results['dec'])  # *u.deg
        idx, d2d, d3d = object_coordinates.match_to_catalog_sky(catalog_coordinates)
        id = eazy_results['id'][idx]
    
    return id

In [ ]:
if 0:
    RA, Dec = 24.3398906, -8.4534802
    object_coordinates = SkyCoord(ra=RA*u.deg, dec=Dec*u.deg)
    catalog_coordinates = SkyCoord(ra=eazy_results['ra'], dec=eazy_results['dec'])  # *u.deg
    idx, d2d, d3d = object_coordinates.match_to_catalog_sky(catalog_coordinates)
    id = eazy_results['id'][idx]
    id

In [ ]:
#fig, data = eazy_run.show_fit(id, xlim=[0.5, 5], show_fnu=True, logpz=True, show_prior=True)
#id

# Show a galaxy (or any object) detected in the image